In [ ]:
from webscraper import Scraper

In [ ]:
scraper = Scraper() #call scraper class
scraper.accept_cookies()
#scraper.navigate()
scraper.search_bar("final fantasy") #add search keyword here

In [ ]:
import selenium
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager #installs Chrome webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

container = scraper.find_container()
'''
#keep scrolling down page until no more results load up - will run properly if there is no other method being called in same cell
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = scraper.driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    scraper.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = scraper.driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height'''

In [ ]:
product_links = scraper.collect_page_links()

In [ ]:
from selenium.common.exceptions import NoSuchElementException
import uuid #universally unique id
import time
from bs4 import BeautifulSoup
import requests
import re
import os, shutil, os.path

#add link and product info to dictionary
product_dict = {"product_link": [], "product_name" :[], "price" : [], "description" : [], "product_id": [], "product_uuid":[]}
image_dict = {"image_link": [], "product_uuid": [], "image_uuid": [], "image": []}

def retrieve_data():
   #link_list #lists all urls in specified webpage section
   
   for link in product_links[:50]: #iterate through first 50 links
      '''
         passes any links which relate to games as DOB is required 
         otherwise any images downloaded from these pages are part of the DOB frame
         --> 50 data entries will not be scraped as a result
      '''
      if "download" in link:
         pass
      elif "remastered" in link:
         pass
      elif "pc" in link:
         pass
      elif "ps4" in link:
         pass
      elif "switch" in link:
         pass
      elif "xbox" in link:
         pass
      elif "dlc" in link:
         pass
      elif "xiii" in link:
         pass
      elif "xii" in link:
         pass
      elif "starter" in link:
         pass
      elif "ps5" in link:
         pass
      elif "mac" in link:
         pass
      elif "vita" in link:
         pass
      else:
         scraper.driver.get(link)
         time.sleep(2)
         product_dict["product_link"].append(link)

         try: #get product name
            product_name = scraper.driver.find_element(By.XPATH, 
               '//*[@id="desktop-wrapper-title"]/div/header/h1')
               #shortened form of //*[@id="responsive-wrapper-title"]/header/h1
               #.//h1[@class="product-title"] - works some of the time
               #//*[@id="desktop-wrapper-title"]/div/header/h1
            product_dict["product_name"].append(product_name.text)
            print("Product name obtained")
         except NoSuchElementException: #not all links accessed will have the same attributes
            product_dict["product_name"].append("N/A")
         
         time.sleep(3)
         try: #get price
            price = scraper.driver.find_element(By.XPATH, '//span[@class = "prices"]')
            #//*[@id="main-content"]/article/div[1]/div/div[2]/div[2]/div[1]/div[1]/div/span/span
               #shortened form of //*[@id="main-content"]/article/div[1]/div/div[2]/div[2]/div[1]/div[1]/div/span
            product_dict["price"].append(price.text)
            print("Price obtained")
         except NoSuchElementException:
            product_dict["price"].append("N/A")
         
         time.sleep(3)
         try: #get product description
            desc = scraper.driver.find_element(By.XPATH, './/div[@class="tab-pane-content"]')
               #shortened form of //*[@id="desc-collapse"]/div/div
            product_dict["description"].append(desc.text)
            print("Product description obtained")
         except NoSuchElementException:
            product_dict["description"].append("N/A")
         
         time.sleep(3)
         try: #get product SKU/ID from URL
            #to get SKU - product details tab needs to be clicked and then an if statement to get the SKU otherwise other info will be obtained
            r = link.rsplit("/", 6) #split link 6 times after every '/'
            product_dict["product_id"].append(r[5])
            print("Product ID obtained")
         except NoSuchElementException:
            product_dict["product_id"].append("N/A")

         time.sleep(3)
         try: #generate V4 UUID for product and image
            product_uuid = uuid.uuid4()
            image_uuid = uuid.uuid4()
            product_dict["product_uuid"].append(product_uuid), image_dict["product_uuid"].append(product_uuid)
            image_dict["image_uuid"].append(image_uuid)

            print("UUID generated")
         except NoSuchElementException:
            product_dict["product_uuid"].append("N/A")
            image_dict["product_uuid"].append("N/A")
            image_dict["image_uuid"].append("N/A")
         
         time.sleep(3)
        
         try: #download and save product image using product ID as file name
            '''
               ensure window is maximised for images to be downloaded successfully (method already implemented)
            '''
            image = f"{r[5]}.png"
            with open(image, "wb") as file: #wb = write and binary mode
               img = scraper.driver.find_element(By.XPATH, 
                  '//img[@class="boxshot lazyloaded"]')
               #//*[@id="main-content"]/article/div[1]/div/div[1]/div[2]/figure/img
               file.write(img.screenshot_as_png)
               time.sleep(3)
            image_dict["image"].append(image)
            print("Image downloaded")
         except:
            image_dict["image"].append("N/A")
            print("Image not downloaded")
         time.sleep(3)

         try:
            html_page = requests.get(link).content #send get request for each link in range and get content of each request
            soup = BeautifulSoup(html_page)
            images = []
            for img in soup.find_all("img"): #finds .jpg images only
               images.append(img.get("data-lazy")) #try "srcset" or "data-srcset" if this doesn't work
                
            #add image link(s) to image_dict
            image_link = images[2] #retrieves image link only
            image_dict["image_link"].append(image_link)
            print("Image link added")
         except:
            image_dict["image_link"].append("N/A")
            print("Image link not added")
         time.sleep(3)
      
retrieve_data()

In [ ]:
import pandas as pd

df_products = pd.DataFrame(product_dict) #displays product dictionary in panda dataframe
df_images = pd.DataFrame(image_dict) #displays image dictionary in panda dataframe

In [ ]:
#df_images
df_products

In [ ]:
import os, os.path
import json
from json import JSONEncoder
from uuid import UUID

create_data_folder = "../webscraping/raw_data"
product_file = "data"
image_file = "image_data"
create_file = os.path.join(create_data_folder, product_file+".json") #add file type here
create_image_file = os.path.join(create_data_folder, image_file+".json")

#Dealing with no UUID serialization support in json
JSONEncoder_olddefault = JSONEncoder.default
def JSONEncoder_newdefault(self, o):
    if isinstance(o, UUID): return str(o)
    return JSONEncoder_olddefault(self, o)
JSONEncoder.default = JSONEncoder_newdefault


try: #create raw_data folder in current directory - check if folder already exists
    if not os.path.exists(create_data_folder): #if folder doesn't exist
        os.mkdir(create_data_folder) 
        with open(create_file, "w") as fp: #specify path here - create data.json file
                json.dump(product_dict, fp,  indent=4)
        with open(create_image_file, "w") as fpi:  #create image_data.json file
                json.dump(image_dict, fpi,  indent=4)

    elif os.path.exists(create_data_folder): #if folder already exists
        with open(create_file, "w") as fp: 
                json.dump(product_dict, fp,  indent=4)
        with open(create_image_file, "w") as fpi: 
                json.dump(image_dict, fpi,  indent=4)
except FileExistsError:
    print("Already exists")

In [ ]:
#move files into a new directory/folder
import os, shutil, os.path #shutil - to save it locally

create_image_folder = "images" #create images folder in current directory
sourcepath = "../webscraping"
source = os.listdir(sourcepath)
destinationpath = "../webscraping/images"

try:
 if not os.path.exists(create_image_folder):
  os.mkdir(create_image_folder)
  for files in source:
      if files.endswith('.png'):
          shutil.move(os.path.join(sourcepath, files), os.path.join(destinationpath, files))  
 elif os.path.exists(create_image_folder):
      for files in source:
        if files.endswith('.png'):
            shutil.move(os.path.join(sourcepath, files), os.path.join(destinationpath, files)) 
except FileExistsError:
 print("Already exists")